In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import pandas as pd
directory = '/content/drive/MyDrive/combined_image_TURE'      # ****이미지 폴더 경로
jpg_files = os.listdir(directory)   # 해당폴더에서 파일 가져옴
file_name_list_jpg = [os.path.splitext(file)[0] for file in jpg_files if file.endswith(".jpg")]# 추가
jpg_files_path_list= ["{}/{}".format(directory,file) for file in jpg_files if file.endswith(".jpg")]

directory = '/content/drive/MyDrive/combined_label_TURE'      # ****라벨링 폴더 경로
txt_files = os.listdir(directory)   # 해당폴더에서 파일 가져옴
file_name_list_txt = [os.path.splitext(file)[0] for file in txt_files if file.endswith(".txt")]# 추가
txt_files_path_list= ["{}/{}".format(directory,file) for file in txt_files if file.endswith(".txt")]

# 텍스트 데이터프레임 
df_txt = pd.DataFrame(zip(file_name_list_txt,txt_files_path_list),
                          columns=['file_name','txt_files_paths'])
# 이미지 데이터프레임 
df_jpg = pd.DataFrame(zip(file_name_list_jpg,jpg_files_path_list),
                 columns=['file_name','jpg_files_paths'])

total_path_df = pd.merge(left = df_txt , right = df_jpg, how = "inner", on = "file_name")

# total_path_df 확인
pd.set_option("display.max_colwidth", 200)
total_path_df

In [ ]:
%cd /content
!git clone https://github.com/ultralytics/yolov5.git  # github에서 yolov5 불러옴(pytorch기반)

In [ ]:
%cd /content/yolov5/
!pip install -r requirements.txt # requirements.txt: 설치할 패키지 관리
                                 #  pip list를 입력하면 다음과 같이 pip으로 설치된 모든 패키지가 나옴
                                 # 이 패키지들을 그대로 다른 환경에서 설치하고 싶을땐 어떻게하면 좋을까?
                                 # 일일이 손으로 타이핑하기엔 너무 귀찮을 것 같다.
                                 # requirements.txt는 이러한 패키지 목록이 나열되어있는 텍스트 파일이다.

In [ ]:
%mkdir /content/dataset
%mkdir /content/dataset/images
%mkdir /content/dataset/labels

import shutil
# yaml(이미지/라벨링경로, class_id에 대한 정보가 저장되어있음)
shutil.copy("/content/drive/MyDrive/멀티캠퍼스/project/custom_1.yaml","/content/dataset") # class, test, val에 대한 데이터 들어있음  
%cat /content/dataset/custom_1.yaml

# images 폴더로 모으기
import shutil
for path in total_path_df['jpg_files_paths']:
  jpg_origin = path
  total_jpg_file = (path.split("/"))[-1]
  jpg_copy = "/content/dataset/images/" + total_jpg_file
  shutil.copy(jpg_origin,jpg_copy)

# labels 폴더로 모으기
for path in total_path_df['txt_files_paths']:
  txt_origin = path
  total_txt_file = (path.split("/"))[-1]
  txt_copy = "/content/dataset/labels/" + total_txt_file
  shutil.copy(txt_origin,txt_copy)

%cd /
from glob import glob # 파이썬에서 디렉토리 내의 파일과 폴더의 이름들을 리스트로 모아서 반환받을 수 있는 glob.glob

img_list = glob('/content/dataset/images/*.jpg')  # 폴더에 있는 이미지 파일이름 리스트로 모아서 반환
print(len(img_list))

from sklearn.model_selection import train_test_split

train_img_list, val_img_list = train_test_split(img_list, test_size=0.2, random_state=2000)
print(len(train_img_list), len(val_img_list))

with open('/content/dataset/train.txt', 'w') as f:  #  train 이미지 리스트 train.txt파일 생성
  f.write('\n'.join(train_img_list) + '\n')
with open('/content/dataset/val.txt', 'w') as f:    #  val 이미지 리스트 train.txt파일 생성
  f.write('\n'.join(val_img_list) + '\n')

import yaml

with open('/content/dataset/custom_1.yaml', 'r') as f:
  data = yaml.safe_load(f)          # yaml.load는 문자열을 파이썬데이터로 바꾸어줌

print(data)   # train(train.txt 경로), val(val.txt 경로), nc(클래스 수), names(클래스 이름)에 대한 정보 보여줌

data['train'] = '/content/dataset/train.txt' # train.txt 경로 지정
data['val'] = '/content/dataset/val.txt'     # val.txt 경로 지정

with open('/content/dataset/custom_1.yaml', 'w') as f:
  yaml.dump(data, f)  # yaml.dump는 파이썬데이터를 yaml형태로 바꾸어줌

print(data)

In [ ]:
%cd /content/yolov5/
# 모델 훈련
!python train.py --img 416 --batch 32 --epochs 50 --data /content/dataset/custom_1.yaml --cfg ./models/yolov5x.yaml --weights yolov5x.pt  --name gun_yolov5x_results # --evolve: hyper

In [ ]:
# 성능 평가
%load_ext tensorboard
%tensorboard --logdir /content/yolov5/runs/

In [ ]:
import os
%cd /content/yolov5/runs/train/
os.rename("gun_yolov5x_results","exp0_gun_yolov5x_results")
shutil.move("/content/yolov5/runs/train/exp0_gun_yolov5x_results", "/content/yolov5/runs/")
%cd /content/yolov5/runs/exp0_gun_yolov5x_results//weights/
os.rename("best.pt","best_gun_yolov5x_results.pt")

In [ ]:
from IPython.display import Image
import os

val_img_path = val_img_list[4]
%cd /content/yolov5/
# 객체 탐지
!python detect.py --weights /content/yolov5/runs/exp0_gun_yolov5x_results/weights/best_gun_yolov5x_results.pt --img 416 --conf 0.5 --save-txt --source "{val_img_path}" # 추가내용 --save-txt : 예측좌표

In [ ]:
# 객체 결과 이미지 확인
Image(os.path.join('/content/yolov5/runs/detect/exp', os.path.basename(val_img_path)))